In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import os
from numba import jit
import scipy
import scipy.stats as stats
import argparse
def bin_log(data, bins=50):
    if(type(bins) is int):
        binning = np.logspace(0,np.log10(np.max(data)), bins)
    else:
        binning = bins
    hist, edges = np.histogram(data,bins=binning, density=True)
    mask = hist > 0
    hist = hist[mask]
    edges = ((edges[0:-1]+edges[1:])*0.5)[mask]
    return hist, edges, binning
def load_params(filename):
    pdcit = {}
    f = open(filename)
    for l in f.readlines():
        a,b = l.split('=')
        pdcit[a] = float(b)
    return pdcit

In [2]:
folder = "vdep_3362809398136477573/"

In [10]:
pms = load_params(folder + 'params.txt')
pms['Lx'] = int(pms['Lx'])
pms['Ly'] = int(pms['Ly'])
print(pms)
print('Loading...')
#df = pd.read_csv(folder + 'data.txt', sep=' ')
#data = df.values
data = np.loadtxt(folder + 'data.txt' , delimiter=' ', skiprows=1)
seq_data = np.loadtxt(folder + 'a_seq.txt' , delimiter=' ', skiprows=1)

{'Lx': 512, 'Ly': 512, 'k0': 0.06, 'k1': 0.0, 'k2': 1.0, 'dh': 0.1, 'f1': 1.0, 'f2': 1.0}
Loading...


In [11]:
print('Filtering (area)...')
max_n_events = 2000

#save all events with at least area min_area
min_area = 100
valid_indices = np.argwhere(np.logical_and(data[:,2] >= min_area, data[:,3]==1)).flatten()
np.savetxt( 'events_to_save_area.txt', valid_indices[:max_n_events], fmt='%i')

Filtering (area)...


In [12]:
print('Filtering (seq)...')

min_length = 50
max_length = 1000
mainshock_idxs = np.argwhere(data[:,3]==0).flatten()
lengths = np.diff(mainshock_idxs)
mask_for_ms = np.argwhere(np.logical_and(lengths >= min_length, lengths <= max_length)).flatten()
good_ms = mainshock_idxs[mask_for_ms]
good_lens = lengths[mask_for_ms]

Filtering (seq)...


In [13]:
#good_events = [ data[idx:idx+ln] for idx, ln in zip(good_ms, good_lens) ]

In [14]:
np.savetxt( 'events_to_save_seq.txt', np.concatenate([ np.arange(idx,idx+ln) for idx, ln in zip(good_ms, good_lens) ]).astype('int')[:max_n_events], fmt='%i')